In [1]:
import requests
from bs4 import BeautifulSoup,SoupStrainer
import pandas as pd
from IPython.display import Markdown, display,HTML,Image
from selenium import webdriver
import time
from pandas.plotting import table 
import matplotlib.pyplot as plt
from html2image import Html2Image
hti = Html2Image(browser='edge',temp_path=r'C:\\Users\\g\\AppData\\Local\\Temp\\html2image\\')

# May have to come up with an alternative to the offset for newer solvers


In [2]:
# get the solution links
pages = ['https://www.janestreet.com/puzzles/archive/index.html',
         'https://www.janestreet.com/puzzles/archive/page2/index.html',
         'https://www.janestreet.com/puzzles/archive/page3/index.html',
         'https://www.janestreet.com/puzzles/archive/page4/index.html',
         'https://www.janestreet.com/puzzles/archive/page5/index.html',
         'https://www.janestreet.com/puzzles/archive/page6/index.html',
         'https://www.janestreet.com/puzzles/archive/page7/index.html',
         'https://www.janestreet.com/puzzles/archive/page8/index.html',
         'https://www.janestreet.com/puzzles/archive/page9/index.html',
         'https://www.janestreet.com/puzzles/archive/page10/index.html',
         'https://www.janestreet.com/puzzles/archive/page11/index.html',
         'https://www.janestreet.com/puzzles/archive/page12/index.html']  
links = ['https://www.janestreet.com/puzzles/current-puzzle/']
for url in pages:
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    links += ['https://www.janestreet.com/'+i['href'] for i in soup.find_all('a', {'class' :'solution-link'})]   

In [8]:
# scrape the names of solvers
participants = []
order = 1
driver = webdriver.Firefox()
for url in links:
    driver.get(url)
    time.sleep(5)
    htmlSource = driver.page_source
    soup = BeautifulSoup(htmlSource, 'html.parser')
    participants += [[str(url).split("/")[-1],i.split(" (")[0],order] for i in soup.find_all('p', {'class' :'correct-submissions'})[0].stripped_strings]
    order +=1

In [113]:
# Work out the scores with an offset. Count as attempted all those since the name appeared on a leaderboard.
# No adjustments for similar names etc
# Borrowed the offset idea from this (much more professionally done) tableau league table 
# https://public.tableau.com/app/profile/heidi.stockton/viz/PuzzlesofJaneStreet/JaneStreet

def df_format(styler,caption=""):
    font_to_use = 'arial'
    styler.format({'Score':'{:.2%}','Solved':'{:.0f}','Attempted':'{:.0f}'})
    styler.set_table_attributes("style='display:inline'")
    styler.hide(axis=0,names=True)
    styler.set_caption(caption)
    styler.set_table_styles([{'selector': 'td', 'props': [('text-align', 'center'),
                                                          ('font-size', '12px') ,
                                                          ('font-family',font_to_use )]},
                             {'selector': '.col_heading', 'props': [('text-align', 'center'),
                                                                    ('width', '30px'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                   ('font-size', '12px'),
                                                                    ('font-family',font_to_use )]},
                             {'selector': '.row_heading', 'props': [('text-align', 'left'),
                                                                    ('background-color', 'slategrey'),
                                                                    ('color', 'white'),
                                                                    ('font-size', '12px'),
                                                                    ('font-family',font_to_use )]},
                             {'selector': 'caption', 'props': [ ('font-size', '18px'),
                                                               ('font-weight','bold'),
                                                               ('text-align', 'left'),
                                                               ('font-family',font_to_use )]},
                            ])
    styler.apply(lambda x: ['background: lightgreen'if x.name == 'gareth owen'  else '' for i in x], axis=1)
    my_name =[{'selector': '.row_heading', 'props': 'color : lightgreen'}]
    styler.set_table_styles({'gareth owen': my_name},axis=1,overwrite=False)
                                                      
    return styler

outputs = []

for newcomer_offset in [5]:
    df = pd.DataFrame(participants,columns = ['puzzle','Name','Order'])
    df['Name']=df['Name'].str.lstrip("1234567890. ")
    df['Name'] = df['Name'].str.lower()
    df2 = df[['Name','Order']].groupby('Name').agg({'Name':['count'],'Order': ['max']})
    df2.columns = [' '.join(col).strip() for col in df2.columns.values]
    df2.columns = ['Solved','Attempts']
    df2['Score']=df2['Solved']/(df2['Attempts']+ newcomer_offset)
    outputs += [df2.sort_values(['Score','Solved'],ascending=False)[:15].style.pipe(df_format,caption="With an offset of "+str(newcomer_offset))._repr_html_()]


display(HTML(outputs[0]))

,Solved,Attempts,Score
evan semet,34,34,87.18%
karl mahlburg,52,58,82.54%
senthil rajasekaran,58,69,78.38%
gareth owen,40,49,74.07%
aaditya raghavan,42,52,73.68%
blaine hill,14,14,73.68%
lazar ilic,37,46,72.55%
danica xiong,15,16,71.43%
calvin pozderac,61,82,70.11%
sean egan,52,76,64.20%


In [115]:
outputs = df2.sort_values('Solved',ascending=False)[:15].style.pipe(df_format,caption="Number of Solutions")._repr_html_()

display(HTML(outputs))

,Solved,Attempts,Score
calvin pozderac,61,82,70.11%
senthil rajasekaran,58,69,78.38%
sean egan,52,76,64.20%
karl mahlburg,52,58,82.54%
keith schneider,47,73,60.26%
aaditya raghavan,42,52,73.68%
heidi stockton,40,95,40.00%
gareth owen,40,49,74.07%
sébastien geeraert,38,67,52.78%
lazar ilic,37,46,72.55%


In [ ]:
import xlwings as xw

sht = xw.Book().sheets[0]
sht.range('a1').options(index=False).value =  df